In [1]:
from keras.models import load_model
import tensorflow as tf
import numpy as np
import pickle as pkl

In [2]:
# currently 1 bi-directional lstm layer followed by a dense layer
class LSTM_network:

    def __init__(self, n_hidden, embedding_dim, n_classes, weights=None, debug=False):
        self.n_hidden = n_hidden
        self.embedding_dim = embedding_dim
        self.n_classes = n_classes
        self.debug = debug

        # model parameters
        if weights is not None:
            self.check_weights(weights)
            self.W_x_fward = tf.constant(weights[0], dtype=tf.float64)
            self.W_h_fward = tf.constant(weights[1], dtype=tf.float64)
            self.b_fward = tf.constant(weights[2], dtype=tf.float64)

            self.W_x_bward = tf.constant(weights[3], dtype=tf.float64)
            self.W_h_bward = tf.constant(weights[4], dtype=tf.float64)
            self.b_bward = tf.constant(weights[5], dtype=tf.float64)

            self.W_dense_fw = tf.constant(weights[6][:self.n_hidden], dtype=tf.float64)
            self.W_dense_bw = tf.constant(weights[6][self.n_hidden:], dtype=tf.float64)
            self.b_dense = tf.constant(weights[7], dtype=tf.float64)
        else:
            self.W_x_fward = tf.constant(np.random.randn(self.embedding_dim, 4 * self.n_hidden))
            self.W_h_fward = tf.constant(np.random.randn(self.n_hidden, 4 * self.n_hidden))
            self.b_fward = tf.constant(np.random.randn(4*self.n_hidden,))

            self.W_x_bward = tf.constant(np.random.randn(self.embedding_dim, 4 * self.n_hidden))
            self.W_h_bward = tf.constant(np.random.randn(self.n_hidden, 4 * self.n_hidden))
            self.b_bward = tf.constant(np.random.randn(4 * self.n_hidden, ))

            self.W_dense_fw = tf.constant(np.random.randn(n_hidden, n_classes))
            self.W_dense_bw = tf.constant(np.random.randn(n_hidden, n_classes))
            self.b_dense = tf.constant(np.random.randn(n_classes))

        # prediction of the net
        self.y_hat = tf.Variable(0., shape=tf.TensorShape(None), dtype=tf.float64, name='y_hat')

        # the following order is from keras. You might have to adjust it if you use different frameworks
        self.idx_i = slice(0, self.n_hidden)
        self.idx_f = slice(self.n_hidden, 2 * self.n_hidden)
        self.idx_c = slice(2 * self.n_hidden, 3 * self.n_hidden)
        self.idx_o = slice(3 * self.n_hidden, 4 * self.n_hidden)

    def check_weights(self, weights):
        assert len(weights) == 8
        assert weights[0].shape == weights[3].shape == (self.embedding_dim, 4 * self.n_hidden)
        assert weights[1].shape == weights[4].shape == (self.n_hidden, 4 * self.n_hidden)
        assert weights[2].shape == weights[5].shape == (4 * self.n_hidden, )
        assert weights[6].shape == (2 * self.n_hidden, self.n_classes)
        assert weights[7].shape == (self.n_classes,)

    # x is batch of embedding vectors (batch_size, embedding_dim)
    @tf.function
    def cell_step(self, x, h_old, c_old, W_x, W_h, b):
        # fward pass
        gate_x = tf.matmul(x, W_x)
        gate_h = tf.matmul(h_old, W_h)
        gate_pre = gate_x + gate_h + b
        gate_post = tf.concat([
                            tf.sigmoid(gate_pre[:, self.idx_i]), tf.sigmoid(gate_pre[:, self.idx_f]),
                            tf.tanh(gate_pre[:, self.idx_c]), tf.sigmoid(gate_pre[:, self.idx_o]),
                            ], axis=1)
        c_new = gate_post[:, self.idx_f] * c_old + gate_post[:, self.idx_i] * gate_post[:, self.idx_c]
        h_new = gate_post[:, self.idx_o] * tf.tanh(c_new)
        return gate_pre, gate_post, c_new, h_new

    # x is batch of embedding vectors (batch_size, embedding_dim)
    @tf.function
    def one_step_fward(self, x, h_old_fw, c_old_fw):
        fward = self.cell_step(x, h_old_fw, c_old_fw, self.W_x_fward, self.W_h_fward, self.b_fward)
        return fward

    # x_rev is batch of embedding vectors (batch_size, embedding_dim)
    @tf.function
    def one_step_bward(self, x_rev, h_old_bw, c_old_bw):
        bward = self.cell_step(x_rev, h_old_bw, c_old_bw, self.W_x_bward, self.W_h_bward, self.b_bward)
        return bward

    # input is full batch (batch_size, T, embedding_dim)
    @tf.function(experimental_relax_shapes=True)
    def full_pass(self, x):
        assert len(x.shape) == 3, '3 dimensional input required, got input of len {}'.format(len(x.shape))
        batch_size = x.shape[0]
        # we have to reorder the input since tf.scan scans the input along the first axis
        elems = tf.transpose(x, perm=[1,0,2])
        initializer = (tf.constant(np.zeros((batch_size, 4 * self.n_hidden))),  # gates_pre
                       tf.constant(np.zeros((batch_size, 4 * self.n_hidden))),  # gates_post
                       tf.constant(np.zeros((batch_size, self.n_hidden))),      # c_t
                       tf.constant(np.zeros((batch_size, self.n_hidden))))      # h_t
        fn_fward = lambda a, x: self.one_step_fward(x, a[3], a[2])
        fn_bward = lambda a, x: self.one_step_bward(x, a[3], a[2])
        # outputs contain tesnors with (T, gates_pre, gates_post, c,h)
        o_fward = tf.scan(fn_fward, elems, initializer=initializer)
        o_bward = tf.scan(fn_bward, elems, initializer=initializer, reverse=True)
        # final prediction scores
        y_fward = tf.matmul(o_fward[3][-1], self.W_dense_fw)
        y_bward = tf.matmul(o_bward[3][0], self.W_dense_bw)
        y_hat = y_fward + y_bward + self.b_dense
        self.y_hat.assign(y_hat)
        return y_hat, o_fward, o_bward

    def lrp_linear_layer(self, h_in, w, b, hout, Rout, bias_nb_units, eps, bias_factor=0.0):
        """
        LRP for a linear layer with input dim D and output dim M.
        Args:
        - hin:            forward pass input, of shape (batch_size, D)
        - w:              connection weights, of shape (D, M)
        - b:              biases, of shape (M,)
        - hout:           forward pass output, of shape (batch_size, M) (unequal to np.dot(w.T,hin)+b if more than
                          one incoming layer!)
        - Rout:           relevance at layer output, of shape (batch_size, M)
        - bias_nb_units:  total number of connected lower-layer units (onto which the bias/stabilizer contribution
                          is redistributed for sanity check)
        - eps:            stabilizer (small positive number)
        - bias_factor:    set to 1.0 to check global relevance conservation, otherwise use 0.0 to ignore
                          bias/stabilizer redistribution (recommended)
        Returns:
        - Rin:            relevance at layer input, of shape (batch_size, D)
        """
        bias_factor_t = tf.constant(bias_factor, dtype=tf.float64)
        eps_t = tf.constant(eps, dtype=tf.float64)
        sign_out = tf.cast(tf.where(hout >= 0, 1., -1.), tf.float64)   # shape (batch_size, M)
        numerator_1 = tf.expand_dims(h_in, axis=2) * w
        numerator_2 = bias_factor_t * (tf.expand_dims(b, 0) + eps_t * sign_out) / bias_nb_units
        # use the following term if you want to check relevance property
        #numerator_2 =  (bias_factor_t * tf.expand_dims(b, 0) + eps_t * sign_out) / bias_nb_units
        numerator = numerator_1 + tf.expand_dims(numerator_2, 1)
        denom = hout + (eps*sign_out)
        message = numerator / tf.expand_dims(denom, 1) * tf.expand_dims(Rout, 1)
        R_in = tf.reduce_sum(message, axis=2)
        return R_in

    def lrp(self, x, y=None, eps=1e-3, bias_factor=0.0):
        """
        LRP for a batch of samples x.
        Args:
        - x:              input array. dim = (batch_size, T, embedding_dim)
        - y:              desired output_class to explain. dim = (batch_size,)
        - eps:            eps value for lrp-eps
        - bias_factor:    bias factor for lrp
        Returns:
        - Relevances:     relevances of each input dimension. dim = (batch_size, T, embedding_dim
        """
        assert len(x.shape) == 3, '3 dimensional input required, got input of len {}'.format(len(x.shape))
        lrp_pass = self.lrp_lstm(x,y,eps, bias_factor)
        # add forward and backward relevances of x.
        # Here we have to reverse R_x_fw since the tf.scan() function starts at the last timestep (T-1) and moves to
        # timestep 0. Therefore the last entry of lrp_pass[2] belongs to the first timestep of x. Likewise, the last
        # entry of lrp_pass[5] (R_x_rev) belongs to the last timestep of x and is thus already in the right order.
        Rx_ = tf.reverse(lrp_pass[2], axis=[0]) + lrp_pass[5]
        Rx = tf.transpose(Rx_, perm=(1,0,2))  # put batch dimension to first dim again
        # remaining relevance is sum of last entry of Rh and Rc
        rest = tf.reduce_sum(lrp_pass[0][-1] + lrp_pass[1][-1] + lrp_pass[3][-1] + lrp_pass[4][-1], axis=1)
        return Rx, rest

    @tf.function
    def lrp_lstm(self, x, y=None, eps=1e-3, bias_factor=0.0):
        batch_size = x.shape[0]
        T = x.shape[1]
        x_rev = tf.reverse(x, axis=[1])
        # update inner states
        y_hat, output_fw, output_bw = self.full_pass(x)
        # if classes are given, use them. Else choose prediction of the network
        if y is not None:
            assert y.shape == (batch_size, )
            if not y.dtype is tf.int64:
                y = tf.cast(y, tf.int64)
            R_out_mask = tf.one_hot(y, depth=self.n_classes, dtype=tf.float64)
        else:
            R_out_mask = tf.one_hot(tf.argmax(y_hat, axis=1), depth=self.n_classes, dtype=tf.float64)
        R_T = y_hat * R_out_mask
        gates_pre_fw, gates_post_fw, c_fw, h_fw = output_fw
        gates_pre_bw, gates_post_bw, c_bw, h_bw = output_bw
        # c and h have one timestep more than x (the initial one, we have to add these zeros manually)
        zero_block = tf.constant(np.zeros((1, batch_size, self.n_hidden)))
        c_fw = tf.concat([c_fw, zero_block], axis=0)
        h_fw = tf.concat([h_fw, zero_block], axis=0)
        gates_pre_bw = tf.reverse(gates_pre_bw, [0])
        gates_post_bw = tf.reverse(gates_post_bw, [0])
        c_bw = tf.reverse(c_bw, [0])
        h_bw = tf.reverse(h_bw, [0])
        c_bw = tf.concat([c_bw, zero_block], axis=0)
        h_bw = tf.concat([h_bw, zero_block], axis=0)

        # first calculate relevaces from final linear layer
        Rh_fw_T = self.lrp_linear_layer(h_fw[T - 1], self.W_dense_fw, self.b_dense,
                                       y_hat, R_T, 2*self.n_hidden, eps, bias_factor)
        Rh_bw_T = self.lrp_linear_layer(h_bw[T - 1], self.W_dense_bw, self.b_dense,
                                       y_hat, R_T, 2*self.n_hidden, eps, bias_factor)
        if self.debug:
            tf.print('Dense: Input relevance', tf.reduce_sum(R_T, axis=1))
            tf.print('Dense: Output relevance', tf.reduce_sum(Rh_fw_T+Rh_bw_T, axis=1))
        elems = np.arange(T-1, -1, -1)
        initializer = (
                       Rh_fw_T,                                                                     # R_h_fw
                       Rh_fw_T,                                                                     # R_c_fw
                       tf.constant(np.zeros((batch_size, self.embedding_dim)), name='R_x_fw'),      # R_x_fw
                       Rh_bw_T,                                                                     # R_h_bw
                       Rh_bw_T,                                                                     # R_c_bw
                       tf.constant(np.zeros((batch_size, self.embedding_dim)), name='R_x_bw')       # R_x_bw
                       )
        eye = tf.eye(self.n_hidden, dtype=tf.float64)
        zeros_hidden = tf.constant(np.zeros((self.n_hidden)))

        @tf.function
        def update(input_tuple, t):
            # t starts with T-1 ; the values we want to update are essentially Rh, Rc and Rx
            # input_tuple is (R_h_fw_t+1, R_c_fw_t+1, R_x_fw_t+1, R_h_bw_t+1, R_h_bw_t+1, R_x_bw_t+1)
            #forward
            Rc_fw_t = self.lrp_linear_layer(gates_post_fw[t, :, self.idx_f] * c_fw[t-1, :], eye, zeros_hidden,
                                               c_fw[t, :],  input_tuple[1], 2*self.n_hidden, eps, bias_factor)
            R_g_fw = self.lrp_linear_layer(gates_post_fw[t,:,self.idx_i] * gates_post_fw[t,:,self.idx_c], eye,
                                        zeros_hidden, c_fw[t, :], input_tuple[1], 2*self.n_hidden, eps, bias_factor)
            if self.debug:
                tf.print('Fw1: Input relevance', tf.reduce_sum(input_tuple[1], axis=1))
                tf.print('Fw1: Output relevance', tf.reduce_sum(Rc_fw_t + R_g_fw, axis=1))
            Rx_t = self.lrp_linear_layer(x[:,t], self.W_x_fward[:, self.idx_c], self.b_fward[self.idx_c],
                                         gates_pre_fw[t, :, self.idx_c], R_g_fw, self.n_hidden + self.embedding_dim, eps, bias_factor)
            Rh_fw_t = self.lrp_linear_layer(h_fw[t-1, :], self.W_h_fward[:, self.idx_c], self.b_fward[self.idx_c],
                                            gates_pre_fw[t, :, self.idx_c], R_g_fw, self.n_hidden + self.embedding_dim, eps, bias_factor
                                            )
            if self.debug:
                tf.print('Fw2: Input relevance', tf.reduce_sum(R_g_fw, axis=1))
                tf.print('Fw2: Output relevance', tf.reduce_sum(Rx_t,axis=1)+tf.reduce_sum(Rh_fw_t, axis=1))
            if t != 0:
                Rc_fw_t += Rh_fw_t
            #backward
            Rc_bw_t = self.lrp_linear_layer(gates_post_bw[t, :, self.idx_f] * c_bw[t-1, :], eye, zeros_hidden,
                                            c_bw[t, :], input_tuple[4], 2*self.n_hidden, eps, bias_factor)
            R_g_bw = self.lrp_linear_layer(gates_post_bw[t, :, self.idx_i] * gates_post_bw[t, :, self.idx_c], eye,
                                           zeros_hidden, c_bw[t,:], input_tuple[4], 2*self.n_hidden, eps, bias_factor)
            if self.debug:
                tf.print('Bw1: Input relevance', tf.reduce_sum(input_tuple[4], axis=1))
                tf.print('Bw1: Output relevance', tf.reduce_sum(Rc_bw_t + R_g_bw, axis=1))
            Rx_rev_t = self.lrp_linear_layer(x_rev[:, t], self.W_x_bward[:, self.idx_c], self.b_bward[self.idx_c],
                                            gates_pre_bw[t, :, self.idx_c], R_g_bw, self.n_hidden + self.embedding_dim, eps, bias_factor)
            Rh_bw_t = self.lrp_linear_layer(h_bw[t-1, :], self.W_h_bward[:, self.idx_c], self.b_bward[self.idx_c],
                                            gates_pre_bw[t, :, self.idx_c], R_g_bw, self.n_hidden + self.embedding_dim, eps, bias_factor
                                            )
            if self.debug:
                tf.print('Bw2: Input relevance', tf.reduce_sum(R_g_bw, axis=1))
                tf.print('Bw2: Output relevance', tf.reduce_sum(Rx_rev_t,axis=1)+tf.reduce_sum(Rh_bw_t, axis=1))
            if t != 0:
                Rc_bw_t += Rh_bw_t
            return Rh_fw_t, Rc_fw_t, Rx_t, Rh_bw_t, Rc_bw_t, Rx_rev_t

        lrp_pass = tf.scan(update, elems, initializer)
        return lrp_pass


In [3]:
# load the keras model and the word-2-vec dictionary. The keras model is trained for the sentiment analysis dataset
# from stanford university and classifies movie reviews into five different categories.
keras_model = load_model('../input/reference-model/sentiment_model.hdf5')
w2v = pkl.load(open('../input/reference-model/token2vec.pkl', 'rb'))
# create the lstm-lrp model
n_hidden = 60
embedding_dim = 60
n_classes = 5
weights = keras_model.get_weights()
print(len(weights))
# our keras model has no bias in the final dense layer. Therefore we add a bias of zero to the weights
weights.append(np.zeros((n_classes,)))
lrp_model = LSTM_network(n_hidden, embedding_dim, n_classes, weights=weights)


7


In [4]:
import codecs
def get_test_sentence(sent_idx):
    """Returns an SST test set sentence and its true label, sent_idx must be an integer in [1, 2210]"""
    idx = 1
    with codecs.open("./data/sequence_test.txt", 'r', encoding='utf8') as f:
        for line in f:
            line          = line.rstrip('\n')
            line          = line.split('\t')
            true_class    = int(line[0])-1         # true class
            words         = line[1].split(' | ')   # sentence as list of words
            if idx == sent_idx:
                return words, true_class
            idx +=1
def get_all_test_sentence():
    res = []
    with codecs.open("../input/sentence-test/sequence_test.txt", 'r', encoding='utf8') as f:
        for line in f:
            line          = line.rstrip('\n')
            line          = line.split('\t')
            true_class    = int(line[0])-1         # true class
            words         = line[1].split(' | ')   # sentence as list of words
            res.append((true_class, words))
    return res

In [5]:
# # for practical example
# eps = 1e-3
# bias_factor = 0.0
# sentence = 'all but the most persnickety preteens should enjoy this nonthreatening but thrilling adventure .'

# # test if the conversion was correct with an example sentence
# # sentence = 'Neither funny nor suspenseful nor particularly well-drawn .'
# tokens = [s.lower() for s in sentence.split()]
# vecs = np.array([w2v[t] for t in tokens])
# y_keras = keras_model.predict(vecs[np.newaxis,:])
# model_prediction_class = np.argmax(y_keras[0])

# # by setting y=None, the relevances will be calculated for the predicted class of the sample. We recommend this
# # usage, however, if you are interested in the relevances towards the 1st class, you could use y = np.array([1])
# explanation, Rest = lrp_model.lrp(vecs[np.newaxis,:], eps=eps, bias_factor=bias_factor)
# # LRP assigns each dimension in the embedding vector a relevance value. To get relevances for each word we can
# # sum up these values
# relevance_list = tf.reduce_sum(explanation, axis=2)[0]
# index_list = sorted(range(len(relevance_list)), key=lambda k: relevance_list[k], reverse=True)
# weighted_score = np.array([0, 0, 0, 0, 0])
# most_related_contribution = np.array([0, 0, 0, 0, 0])
# fine_tuning = np.array([0, 0, 0, 0, 0])
# for index in index_list:
#     word = tokens[index]
#     vecs = np.array([w2v[t] for t in [word]])
#     single_prediction_class = np.argmax(keras_model.predict(vecs[np.newaxis,:])[0])
#     new = np.array([0, 0, 0, 0, 0])
#     new[single_prediction_class] = 1
#     relevance = relevance_list[index]
#     print('word: {}, score: {}'.format(word, new))
#     print('{0:>13}:   {1:8.2f}'.format(word, relevance))
#     contribution_score = relevance*new
#     print('{}:   {}'.format(word, contribution_score))
#     weighted_score = np.sum([weighted_score, contribution_score], axis=0)
#     if index == index_list[0]:
#         most_related_contribution = contribution_score
# most_related_prediction_class = np.argmax(most_related_contribution)
# weighted_prediction_class = np.argmax(weighted_score)
# if (weighted_prediction_class == 0 and most_related_prediction_class == 1) or  (weighted_prediction_class == 4 and most_related_prediction_class == 3):
#     lrp_class = weighted_prediction_class
# else:
#     lrp_class = most_related_prediction_class

# print('most_related_contribution:{}'.format(most_related_contribution))
# print('most_related_prediction_class:{}'.format(most_related_prediction_class))
# print('weighted_score:{}'.format(weighted_score))
# print('weighted_prediction_class:{}'.format(weighted_prediction_class))
# print('lrp prediction class:{}'.format(weighted_prediction_class))


In [6]:
# calculate the recall and precision.'
eps = 1e-3
bias_factor = 0.0
weighted_correct_count = 0
weighted_n_to_p = 0
weighted_p_to_n = 0
weighted_neutral_floating_1 = 0
most_related_correct_count = 0
most_related_n_to_p = 0
most_related_p_to_n = 0
most_related_neutral_floating_1 = 0


zero_to_zero = 0
zero_to_one = 0
zero_to_two = 0
zero_to_three = 0
zero_to_four = 0

one_to_zero = 0
one_to_one = 0
one_to_two = 0
one_to_three = 0
one_to_four = 0


two_to_zero = 0
two_to_one = 0
two_to_two = 0
two_to_three = 0
two_to_four = 0


three_to_zero = 0
three_to_one = 0
three_to_two = 0
three_to_three = 0
three_to_four = 0

four_to_zero = 0
four_to_one = 0
four_to_two = 0
four_to_three = 0
four_to_four = 0

floating_correct_count = 0
floating_n_to_p = 0
floating_p_to_n = 0
floating_neutral_floating_1 = 0

res = get_all_test_sentence()
for i in range(2210):
    true_class, words = res[i]
    tokens = [s.lower() for s in words]
    vecs = np.array([w2v[t] for t in tokens])
    y_keras = keras_model.predict(vecs[np.newaxis,:])
    model_prediction_class = np.argmax(y_keras[0])

    # by setting y=None, the relevances will be calculated for the predicted class of the sample. We recommend this
    # usage, however, if you are interested in the relevances towards the 1st class, you could use y = np.array([1])
    explanation, Rest = lrp_model.lrp(vecs[np.newaxis,:], eps=eps, bias_factor=bias_factor)
    # LRP assigns each dimension in the embedding vector a relevance value. To get relevances for each word we can
    # sum up these values
    relevance_list = tf.reduce_sum(explanation, axis=2)[0]
    index_list = sorted(range(len(relevance_list)), key=lambda k: relevance_list[k], reverse=True)
    weighted_score = np.array([0, 0, 0, 0, 0])
    most_related_contribution = np.array([0, 0, 0, 0, 0])
    fine_tuning = np.array([0, 0, 0, 0, 0])
    for index in index_list:
        word = tokens[index]
        vecs = np.array([w2v[t] for t in [word]])
        single_prediction_class = np.argmax(keras_model.predict(vecs[np.newaxis,:])[0])
        new = np.array([0, 0, 0, 0, 0])
        new[single_prediction_class] = 1
        relevance = relevance_list[index]
        print('word: {}, score: {}'.format(word, new))
        print('{0:>13}:   {1:8.2f}'.format(word, relevance))
        contribution_score = relevance*new
        weighted_score = np.sum([weighted_score, contribution_score], axis=0)
        if index == index_list[0]:
            most_related_contribution = contribution_score
    most_related_prediction_class = np.argmax(most_related_contribution)
    weighted_prediction_class = np.argmax(weighted_score)
    
    
    if (weighted_prediction_class == 0 and most_related_prediction_class == 1) or  (weighted_prediction_class == 4 and most_related_prediction_class == 3):
        lrp_class = weighted_prediction_class
    else:
        lrp_class = most_related_prediction_class
            
    if model_prediction_class == 0:
        if weighted_prediction_class == 0:
            zero_to_zero += 1
        elif weighted_prediction_class == 1:
            zero_to_one += 1
        elif weighted_prediction_class == 2:
            zero_to_two += 1
        elif weighted_prediction_class == 3:
            zero_to_three += 1
        elif weighted_prediction_class == 4:
            zero_to_four += 1
    elif model_prediction_class == 1:
        if weighted_prediction_class == 0:
            one_to_zero += 1
        elif weighted_prediction_class == 1:
            one_to_one += 1
        elif weighted_prediction_class == 2:
            one_to_two += 1
        elif weighted_prediction_class == 3:
            one_to_three += 1
        elif weighted_prediction_class == 4:
            one_to_four += 1
    elif model_prediction_class == 2:
        if weighted_prediction_class == 0:
            two_to_zero += 1
        elif weighted_prediction_class == 1:
            two_to_one += 1
        elif weighted_prediction_class == 2:
            two_to_two += 1
        elif weighted_prediction_class == 3:
            two_to_three += 1
        elif weighted_prediction_class == 4:
            two_to_four += 1
    elif model_prediction_class == 3:
        if weighted_prediction_class == 0:
            three_to_zero += 1
        elif weighted_prediction_class == 1:
            three_to_one += 1
        elif weighted_prediction_class == 2:
            three_to_two += 1
        elif weighted_prediction_class == 3:
            three_to_three += 1
        elif weighted_prediction_class == 4:
            three_to_four += 1
    elif model_prediction_class == 4:
        if weighted_prediction_class == 0:
            four_to_zero += 1
        elif weighted_prediction_class == 1:
            four_to_one += 1
        elif weighted_prediction_class == 2:
            four_to_two += 1
        elif weighted_prediction_class == 3:
            four_to_three += 1
        elif weighted_prediction_class == 4:
            four_to_four += 1
    


        
        
#     if model_prediction_class == 0:
#         if weighted_prediction_class == 0:
#             zero_to_zero += 1
#         elif weighted_prediction_class == 1:
#             zero_to_one += 1
#         elif weighted_prediction_class == 2:
#             zero_to_two += 1
#         elif weighted_prediction_class == 3:
#             zero_to_three += 1
#         elif weighted_prediction_class == 4:
#             zero_to_four += 1
#     elif model_prediction_class == 1:
#         if weighted_prediction_class == 0:
#             one_to_zero += 1
#         elif weighted_prediction_class == 1:
#             one_to_one += 1
#         elif weighted_prediction_class == 2:
#             one_to_two += 1
#         elif weighted_prediction_class == 3:
#             one_to_three += 1
#         elif weighted_prediction_class == 4:
#             one_to_four += 1
#     elif model_prediction_class == 2:
#         if weighted_prediction_class == 0:
#             two_to_zero += 1
#         elif weighted_prediction_class == 1:
#             two_to_one += 1
#         elif weighted_prediction_class == 2:
#             two_to_two += 1
#         elif weighted_prediction_class == 3:
#             two_to_three += 1
#         elif weighted_prediction_class == 4:
#             two_to_four += 1
#     elif model_prediction_class == 3:
#         if weighted_prediction_class == 0:
#             three_to_zero += 1
#         elif weighted_prediction_class == 1:
#             three_to_one += 1
#         elif weighted_prediction_class == 2:
#             three_to_two += 1
#         elif weighted_prediction_class == 3:
#             three_to_three += 1
#         elif weighted_prediction_class == 4:
#             three_to_four += 1
#     elif model_prediction_class == 4:
#         if weighted_prediction_class == 0:
#             four_to_zero += 1
#         elif weighted_prediction_class == 1:
#             four_to_one += 1
#         elif weighted_prediction_class == 2:
#             four_to_two += 1
#         elif weighted_prediction_class == 3:
#             four_to_three += 1
#         elif weighted_prediction_class == 4:
#             four_to_four += 1
        
#     if model_prediction_class == 0:
#         if most_related_prediction_class == 0:
#             zero_to_zero += 1
#         elif most_related_prediction_class == 1:
#             zero_to_one += 1
#         elif most_related_prediction_class == 2:
#             zero_to_two += 1
#         elif most_related_prediction_class == 3:
#             zero_to_three += 1
#         elif most_related_prediction_class == 4:
#             zero_to_four += 1
#     elif model_prediction_class == 1:
#         if most_related_prediction_class == 0:
#             one_to_zero += 1
#         elif most_related_prediction_class == 1:
#             one_to_one += 1
#         elif most_related_prediction_class == 2:
#             one_to_two += 1
#         elif most_related_prediction_class == 3:
#             one_to_three += 1
#         elif most_related_prediction_class == 4:
#             one_to_four += 1
#     elif model_prediction_class == 2:
#         if most_related_prediction_class == 0:
#             two_to_zero += 1
#         elif most_related_prediction_class == 1:
#             two_to_one += 1
#         elif most_related_prediction_class == 2:
#             two_to_two += 1
#         elif most_related_prediction_class == 3:
#             two_to_three += 1
#         elif most_related_prediction_class == 4:
#             two_to_four += 1
#     elif model_prediction_class == 3:
#         if most_related_prediction_class == 0:
#             three_to_zero += 1
#         elif most_related_prediction_class == 1:
#             three_to_one += 1
#         elif most_related_prediction_class == 2:
#             three_to_two += 1
#         elif most_related_prediction_class == 3:
#             three_to_three += 1
#         elif most_related_prediction_class == 4:
#             three_to_four += 1
#     elif model_prediction_class == 4:
#         if most_related_prediction_class == 0:
#             four_to_zero += 1
#         elif most_related_prediction_class == 1:
#             four_to_one += 1
#         elif most_related_prediction_class == 2:
#             four_to_two += 1
#         elif most_related_prediction_class == 3:
#             four_to_three += 1
#         elif most_related_prediction_class == 4:
#             four_to_four += 1
    
# # #     if true_class == 0:
# # #         if model_prediction_class == 0:
# # #             zero_to_zero += 1
# # #         elif model_prediction_class == 1:
# # #             zero_to_one += 1
# # #         elif model_prediction_class == 2:
# # #             zero_to_two += 1
# # #         elif model_prediction_class == 3:
# # #             zero_to_three += 1
# # #         elif model_prediction_class == 4:
# # #             zero_to_four += 1
# # #     elif true_class == 1:
# # #         if model_prediction_class == 0:
# # #             one_to_zero += 1
# # #         elif model_prediction_class == 1:
# # #             one_to_one += 1
# # #         elif model_prediction_class == 2:
# # #             one_to_two += 1
# # #         elif model_prediction_class == 3:
# # #             one_to_three += 1
# # #         elif model_prediction_class == 4:
# # #             one_to_four += 1
# # #     elif true_class == 2:
# # #         if model_prediction_class == 0:
# # #             two_to_zero += 1
# # #         elif model_prediction_class == 1:
# # #             two_to_one += 1
# # #         elif model_prediction_class == 2:
# # #             two_to_two += 1
# # #         elif model_prediction_class == 3:
# # #             two_to_three += 1
# # #         elif model_prediction_class == 4:
# # #             two_to_four += 1
# # #     elif true_class == 3:
# # #         if model_prediction_class == 0:
# # #             three_to_zero += 1
# # #         elif model_prediction_class == 1:
# # #             three_to_one += 1
# # #         elif model_prediction_class == 2:
# # #             three_to_two += 1
# # #         elif model_prediction_class == 3:
# # #             three_to_three += 1
# # #         elif model_prediction_class == 4:
# # #             three_to_four += 1
# # #     elif true_class == 4:
# # #         if model_prediction_class == 0:
# # #             four_to_zero += 1
# # #         elif model_prediction_class == 1:
# # #             four_to_one += 1
# # #         elif model_prediction_class == 2:
# # #             four_to_two += 1
# # #         elif model_prediction_class == 3:
# # #             four_to_three += 1
# # #         elif model_prediction_class == 4:
# # #             four_to_four += 1
    if weighted_prediction_class == model_prediction_class:
        weighted_correct_count += 1
    elif weighted_prediction_class in (3, 4) and model_prediction_class in (0, 1):
        weighted_n_to_p += 1
    elif weighted_prediction_class in (0, 1) and model_prediction_class in (3, 4):
        weighted_p_to_n += 1
    elif model_prediction_class == 2 and abs(weighted_prediction_class - model_prediction_class) ==1:
        weighted_neutral_floating_1 += 1
    

print('zero_to_zero:{}'.format(zero_to_zero))
print('zero_to_one:{}'.format(zero_to_one))
print('zero_to_two:{}'.format(zero_to_two))
print('zero_to_three:{}'.format(zero_to_three))
print('zero_to_four:{}'.format(zero_to_four))

print('one_to_zero:{}'.format(one_to_zero))
print('one_to_one:{}'.format(one_to_one))
print('one_to_two:{}'.format(one_to_two))
print('one_to_three:{}'.format(one_to_three))
print('one_to_four:{}'.format(one_to_four))


print('two_to_zero:{}'.format(two_to_zero))
print('two_to_one:{}'.format(two_to_one))
print('two_to_two:{}'.format(two_to_two))
print('two_to_three:{}'.format(two_to_three))
print('two_to_four:{}'.format(two_to_four))


print('three_to_zero:{}'.format(three_to_zero))
print('three_to_one:{}'.format(three_to_one))
print('three_to_two:{}'.format(three_to_two))
print('three_to_three:{}'.format(three_to_three))
print('three_to_four:{}'.format(three_to_four))

print('four_to_zero:{}'.format(four_to_zero))
print('four_to_one:{}'.format(four_to_one))
print('four_to_two:{}'.format(four_to_two))
print('four_to_three:{}'.format(four_to_three))
print('four_to_four:{}'.format(four_to_four))
        
print('weighted_correct_count:{}'.format(weighted_correct_count))
print('weighted_n_to_p:{}'.format(weighted_n_to_p))
print('weighted_p_to_n:{}'.format(weighted_p_to_n))
print('weighted_neutral_floating_1:{}'.format(weighted_neutral_floating_1))
# # # print('most_related_correct_count:{}'.format(most_related_correct_count))
# # # print('most_related_n_to_p:{}'.format(most_related_n_to_p))
# # # print('most_related_p_to_n:{}'.format(most_related_p_to_n))
# # # print('most_related_neutral_floating_1:{}'.format(most_related_neutral_floating_1))


        
    
    


word: ridiculous, score: [0 1 0 0 0]
   ridiculous:       4.32
word: ., score: [0 0 1 0 0]
            .:      -0.05
word: ., score: [0 0 1 0 0]
            .:       0.01
word: hopkins, score: [0 0 0 1 0]
      hopkins:      -0.16
word: brimful, score: [0 0 1 0 0]
      brimful:       0.14
word: ., score: [0 0 1 0 0]
            .:      -0.02
word: hmm, score: [0 0 1 0 0]
          hmm:       0.04
word: ., score: [0 0 1 0 0]
            .:      -0.02
word: immediately, score: [0 0 1 0 0]
  immediately:       0.53
word: ., score: [0 0 1 0 0]
            .:      -0.06
word: ., score: [0 0 1 0 0]
            .:       0.00
word: no., score: [0 1 0 0 0]
          no.:      -0.13
word: boring, score: [1 0 0 0 0]
       boring:       4.20
word: extremely, score: [0 0 1 0 0]
    extremely:       1.00
word: ., score: [0 0 1 0 0]
            .:      -0.01
word: confusing, score: [0 1 0 0 0]
    confusing:       1.99
word: extremely, score: [0 0 1 0 0]
    extremely:       0.85
word: ., score: [0